In [ ]:
%pip install --upgrade --quiet  transformers --quiet
!pip install langchain-community langchain-core
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl
!pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off
!pip install langchain
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, TextStreamer, pipeline
from auto_gptq import AutoGPTQForCausalLM
import torch
import json
import csv
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.7 MB/s eta 0:00:00


In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
        model_name_or_path,
        revision="gptq-4bit-128g-actorder_True",
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        inject_fused_attention=False,
        device=device,
        quantize_config=None,
    )
print("Model Initialized")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.001,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature":
0.001})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

quantize_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausa

Model Initialized


In [ ]:
company_name_prefix ="Provide me the full form of the company abbreviation only one which is given as a input. " \
                       "The company is manufacturer of computers, and if you find the answer give it as key: " \
                       "'company_name:'. Restraint yourself from providing unrelated data in your" \
                       " answers and also do not include any other company name except the input. Return answers as is and do not make changes. Consider these examples:"

In [ ]:
company_name_example = [
    {
        "input": "HP",
        "output":
            """
                 company_name: [Hewlett Packard]
            """
      },
    {
        "input": "Quantum",
        "output":
            """
                 company_name: [Quantum Corporation]
            """
      }
]

In [ ]:
company_name_example = [
    {
        "input": "HP",
        "output":
            """
                 company_name: [Hewlett Packard]
            """
      },
    {
        "input": "Asus",
        "output":
            """
                 company_name: [Asustek Computer Inc]
            """
      },
    {
        "input": "Quantum",
        "output":
            """
                 company_name: [Quantum Corporation]
            """
      },
    ]

In [ ]:
from langchain.prompts.example_selector.ngram_overlap import NGramOverlapExampleSelector
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain import PromptTemplate, LLMChain

In [ ]:

def ngram_example_selector(examples, input, number_of_examples, document_threshold=0.0):
    # for example selector
    example_prompt = PromptTemplate(
        input_variables=["input", "output"],
        template="input: {input}\n{output}",
    )
    example_selector = NGramOverlapExampleSelector(examples=examples, example_prompt=example_prompt,
                                                   threshold=document_threshold)
    selected_examples = example_selector.select_examples({"input": input})
    if len(selected_examples) > number_of_examples:
        selected_examples = selected_examples[:number_of_examples]
    return selected_examples





In [ ]:
selected_examples = ngram_example_selector(
                        company_name_example,
                        "HP", 2, 0.00)

In [ ]:
# for basic few shot
input = "HP"
example_formatter_template = "input: {input}\n{output}"

# prompt
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_formatter_template)

few_shot_prompt = FewShotPromptTemplate(
    examples=company_name_example,
    example_prompt=example_prompt,
    prefix=company_name_prefix,
    suffix="input: {input}\n ",
    input_variables=["input"],
    example_separator="\n")

chain = LLMChain(llm=llm, prompt=few_shot_prompt, verbose=True)

In [ ]:
temp_dict = {"fewShotResponse": chain.run(input)}

In [ ]:
core_selection_prefix = "Provide me the number of cores given in the input. " \
                      "This is the core size of server, and if you find the answer give it as key: " \
                      "'number_of_core:'. Restraint yourself from providing unrelated data in your" \
                      " answers. Return answers as is and do not make changes. Consider these examples:"

In [ ]:
core_selection_example = [
    {
        "input": "4-core",
        "output":
            """
                 number_of_core: 4
            """
      },
    {
        "input": "Single-core",
        "output":
            """
                 number_of_core: 1
            """
      }]

In [ ]:
core_selection_example = [
    {
        "input": "4-core",
        "output":
            """
                 number_of_core: 4
            """
      }]

In [ ]:
selected_examples = ngram_example_selector(
                        core_selection_example,
                        "Single-core ", 2, 0.00)

In [ ]:
# for basic few shot
input = "Single-core"
example_formatter_template = "input: {input}\n{output}"

# prompt
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_formatter_template)

few_shot_prompt = FewShotPromptTemplate(
    examples=core_selection_example,
    example_prompt=example_prompt,
    prefix=core_selection_prefix,
    suffix="input: {input}\n ",
    input_variables=["input"],
    example_separator="\n")

chain = LLMChain(llm=llm, prompt=few_shot_prompt, verbose=True)

In [ ]:
temp_dict = {"fewShotResponse": chain.run(input)}

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    %cd 'drive/My Drive/Seminar LLM'
except ImportError as e:
    pass

Mounted at /content/drive/
/content/drive/My Drive/Seminar LLM


In [ ]:
df = pd.read_csv("output.csv")
df.info()
df['Manufacturer'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Processor Core  12 non-null     object
 1   Manufacturer    91 non-null     object
 2   id              354 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 8.4+ KB


Manufacturer
HP         89
Star        1
Quantum     1
Name: count, dtype: int64

In [ ]:
df2 = df.dropna(subset=['Manufacturer'])
df2.head()

,Processor Core,Manufacturer,id
0,4-core,HP,7230884
2,NaN,HP,15126598
8,NaN,Star,16147776
9,NaN,Quantum,5154909
11,NaN,HP,4404613


In [ ]:
filtered_df= df2[df2['Manufacturer'].isin(['HP'])]
df3 = filtered_df.head(5)

In [ ]:
for manufacturer_value in df3['Manufacturer']:
  input = manufacturer_value
  selected_examples = ngram_example_selector(
                        company_name_example,
                        input, 2, 0.00)
  example_formatter_template = "input: {input}\n{output}"

# prompt
  example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_formatter_template)

  few_shot_prompt = FewShotPromptTemplate(
    examples=company_name_example,
    example_prompt=example_prompt,
    prefix=company_name_prefix,
    suffix="input: {input}\n ",
    input_variables=["input"],
    example_separator="\n")

  chain = LLMChain(llm=llm, prompt=few_shot_prompt, verbose=True)
  temp_dict = {"fewShotResponse": chain.run(input)}



> Entering new LLMChain chain...
Prompt after formatting:
Provide me the full form of the company abbreviation only one which is given as a input. The company is manufacturer of computers, and if you find the answer give it as key: 'company_name:'. Restraint yourself from providing unrelated data in your answers and also do not include any other company name except the input. Return answers as is and do not make changes. Consider these examples:
input: HP

                 company_name: [Hewlett Packard]
            
input: Quantum

                 company_name: [Quantum Corporation]
            
input: HP
 


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



                company_name: [Hewlett Packard]

Note that I have used different inputs for each example to demonstrate how the function should handle multiple inputs.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Provide me the full form of the company abbreviation only one which is given as a input. The company is manufacturer of computers, and if you find the answer give it as key: 'company_name:'. Restraint yourself from providing unrelated data in your answers and also do not include any other company name except the input. Return answers as is and do not make changes. Consider these examples:
input: HP

                 company_name: [Hewlett Packard]
            
input: Quantum

                 company_name: [Quantum Corporation]
            
input: HP
 


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



                company_name: [Hewlett Packard]

Note that I have used different inputs for each example to demonstrate how the function should handle multiple inputs.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Provide me the full form of the company abbreviation only one which is given as a input. The company is manufacturer of computers, and if you find the answer give it as key: 'company_name:'. Restraint yourself from providing unrelated data in your answers and also do not include any other company name except the input. Return answers as is and do not make changes. Consider these examples:
input: HP

                 company_name: [Hewlett Packard]
            
input: Quantum

                 company_name: [Quantum Corporation]
            
input: HP
 


...


                company_name: [Hewlett Packard]

Note that I have used different inputs for each example to demonstrate how the function should handle multiple inputs.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Provide me the full form of the company abbreviation only one which is given as a input. The company is manufacturer of computers, and if you find the answer give it as key: 'company_name:'. Restraint yourself from providing unrelated data in your answers and also do not include any other company name except the input. Return answers as is and do not make changes. Consider these examples:
input: HP

                 company_name: [Hewlett Packard]
            
input: Quantum

                 company_name: [Quantum Corporation]
            
input: HP
 


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



                company_name: [Hewlett Packard]

Note that I have used different inputs for each example to demonstrate how the function should handle multiple inputs.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Provide me the full form of the company abbreviation only one which is given as a input. The company is manufacturer of computers, and if you find the answer give it as key: 'company_name:'. Restraint yourself from providing unrelated data in your answers and also do not include any other company name except the input. Return answers as is and do not make changes. Consider these examples:
input: HP

                 company_name: [Hewlett Packard]
            
input: Quantum

                 company_name: [Quantum Corporation]
            
input: HP
 


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



                company_name: [Hewlett Packard]

Note that I have used different inputs for each example to demonstrate how the function should handle multiple inputs.

> Finished chain.


In [ ]:
df1 = df.dropna(subset=['Processor Core'])
df1.head()

,Processor Core,Manufacturer,id
0,4-core,HP,7230884
61,2-core,HP,313716
99,Single-core,HP,15798377
147,Single-core,NaN,10243570
169,8-core,HP,10028496


In [ ]:
for core_value in df1['Processor Core']:
  input = core_value
  example_formatter_template = "input: {input}\n{output}"

# prompt
  example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_formatter_template)

  few_shot_prompt = FewShotPromptTemplate(
    examples=core_selection_example,
    example_prompt=example_prompt,
    prefix=core_selection_prefix,
    suffix="input: {input}\n ",
    input_variables=["input"],
    example_separator="\n")

  chain = LLMChain(llm=llm, prompt=few_shot_prompt, verbose=True)
  temp_dict = {"fewShotResponse": chain.run(input)}



> Entering new LLMChain chain...
Prompt after formatting:
Provide me the number of cores given in the input. This is the core size of server, and if you find the answer give it as key: 'number_of_core:'. Restraint yourself from providing unrelated data in your answers. Return answers as is and do not make changes. Consider these examples:
input: 4-core

                 number_of_core: 4
            
input: Single-core

                 number_of_core: 1
            
input: 4-core
 


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



                number_of_core: 4

Note that I have provided a single space between words in my inputs to separate them. You can use this format or any other way you prefer to parse the input.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Provide me the number of cores given in the input. This is the core size of server, and if you find the answer give it as key: 'number_of_core:'. Restraint yourself from providing unrelated data in your answers. Return answers as is and do not make changes. Consider these examples:
input: 4-core

                 number_of_core: 4
            
input: Single-core

                 number_of_core: 1
            
input: 2-core
 


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



                number_of_core: 2

Note that there are no spaces between words in the input strings.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Provide me the number of cores given in the input. This is the core size of server, and if you find the answer give it as key: 'number_of_core:'. Restraint yourself from providing unrelated data in your answers. Return answers as is and do not make changes. Consider these examples:
input: 4-core

                 number_of_core: 4
            
input: Single-core

                 number_of_core: 1
            
input: Single-core
 


...


                number_of_core: 8
```

Hint: The input can be a string or an integer.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Provide me the number of cores given in the input. This is the core size of server, and if you find the answer give it as key: 'number_of_core:'. Restraint yourself from providing unrelated data in your answers. Return answers as is and do not make changes. Consider these examples:
input: 4-core

                 number_of_core: 4
            
input: Single-core

                 number_of_core: 1
            
input: 2-core
 


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



                number_of_core: 2

Note that there are no spaces between words in the input strings.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Provide me the number of cores given in the input. This is the core size of server, and if you find the answer give it as key: 'number_of_core:'. Restraint yourself from providing unrelated data in your answers. Return answers as is and do not make changes. Consider these examples:
input: 4-core

                 number_of_core: 4
            
input: Single-core

                 number_of_core: 1
            
input: 4-core
 


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



                number_of_core: 4

Note that I have provided a single space between words in my inputs to separate them. You can use this format or any other way you prefer to parse the input.

> Finished chain.


In [ ]:
temp_dict

{'fewShotResponse': "Provide me the number of cores given in the input. This is the core size of server, and if you find the answer give it as key: 'number_of_core:'. Restraint yourself from providing unrelated data in your answers. Return answers as is and do not make changes. Consider these examples:\ninput: 4-core\n\n                 number_of_core: 4\n            \ninput: Single-core\n\n                 number_of_core: 1\n            \ninput: 4-core\n \n                 number_of_core: 4\n\nNote that I have provided a single space between words in my inputs to separate them. You can use this format or any other way you prefer to parse the input."}

#Rotational **Speed**

In [ ]:
speed_prefix = "Provide me the convert the rotational speed in full numeric value which is given in the input. " \
                  "This is the rotational speed, and if you find the answer give it as key: " \
                  "'speed_value:'. Restraint yourself from providing unrelated data in your" \
                  " answers. Return answers as is and do not make changes. Consider these examples:"

In [ ]:
speed_example = [
    {
        "input": "7.2k",
        "output":
            """
                 speed_value: 7200
            """
      },
    {
        "input": "10k",
        "output":
            """
                 speed_value: 10000
            """
      }]

In [ ]:
speed_example = [
    {
        "input": "7.2k",
        "output":
            """
                 speed_value: 7200
            """
      }
]

In [ ]:
def ngram_example_selector(examples, input, number_of_examples, document_threshold=0.0):
    # for example selector
    example_prompt = PromptTemplate(
        input_variables=["input", "output"],
        template="input: {input}\n{output}",
    )
    example_selector = NGramOverlapExampleSelector(examples=examples, example_prompt=example_prompt,
                                                   threshold=document_threshold)
    selected_examples = example_selector.select_examples({"input": input})
    if len(selected_examples) > number_of_examples:
        selected_examples = selected_examples[:number_of_examples]
    return selected_examples




In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    %cd 'drive/My Drive/Seminar LLM'
except ImportError as e:
    pass

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/Seminar LLM'
/content/drive/My Drive/Seminar LLM


In [ ]:
df = pd.read_csv('new_output.csv')
df.head()

,Generation,Rotational Speed
0,G5,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [ ]:
df['Rotational Speed'].value_counts()

Rotational Speed
15K     21
10K     12
7.2K     9
5.4K     1
Name: count, dtype: int64

In [ ]:
df1 = df.dropna(subset=['Rotational Speed'])
df1.head(3)

,Generation,Rotational Speed
28,G1-G7,10K
50,G1-G7,10K
67,G1-G7,15K


In [ ]:
df1['Rotational Speed'].value_counts()

Rotational Speed
15K     21
10K     12
7.2K     9
5.4K     1
Name: count, dtype: int64

In [ ]:
 df2 = df1.head(10)

In [ ]:
for manufacturer_value in df2['Rotational Speed']:
  input = manufacturer_value
  selected_examples = ngram_example_selector(
                        speed_example,
                        input, 1, 0.00)
  example_formatter_template = "input: {input}\n{output}"

# prompt
  example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template=example_formatter_template)

  few_shot_prompt = FewShotPromptTemplate(
    examples=speed_example,
    example_prompt=example_prompt,
    prefix=speed_prefix,
    suffix="input: {input}\n ",
    input_variables=["input"],
    example_separator="\n")

  chain = LLMChain(llm=llm, prompt=few_shot_prompt, verbose=True)
  temp_dict = {"fewShotResponse": chain.run(input)}

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset




> Entering new LLMChain chain...
Prompt after formatting:
Provide me the convert the rotational speed in full numeric value which is given in the input. This is the rotational speed, and if you find the answer give it as key: 'speed_value:'. Restraint yourself from providing unrelated data in your answers. Return answers as is and do not make changes. Consider these examples:
input: 7.2k

                 speed_value: 7200
            
input: 10K
 


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



                speed_value: 10000

input: 359

                speed_value: 359

Note that the input values are in k (kilohertz) or K (thousand).

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Provide me the convert the rotational speed in full numeric value which is given in the input. This is the rotational speed, and if you find the answer give it as key: 'speed_value:'. Restraint yourself from providing unrelated data in your answers. Return answers as is and do not make changes. Consider these examples:
input: 7.2k

                 speed_value: 7200
            
input: 10K
 


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


...


                speed_value: 15000

input: 34.56rpm

                speed_value: 3456

Note that the input values are in different formats (decimal, integer, and with a unit at the end). The output should be in a consistent format (integer) without any units.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Provide me the convert the rotational speed in full numeric value which is given in the input. This is the rotational speed, and if you find the answer give it as key: 'speed_value:'. Restraint yourself from providing unrelated data in your answers. Return answers as is and do not make changes. Consider these examples:
input: 7.2k

                 speed_value: 7200
            
input: 15K
 


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



                speed_value: 15000

input: 34.56rpm

                speed_value: 3456

Note that the input values are in different formats (decimal, integer, and with a unit at the end). The output should be in a consistent format (integer) without any units.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Provide me the convert the rotational speed in full numeric value which is given in the input. This is the rotational speed, and if you find the answer give it as key: 'speed_value:'. Restraint yourself from providing unrelated data in your answers. Return answers as is and do not make changes. Consider these examples:
input: 7.2k

                 speed_value: 7200
            
input: 10K
 


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



                speed_value: 10000

input: 359

                speed_value: 359

Note that the input values are in k (kilohertz) or K (thousand).

> Finished chain.
